In [17]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0


In [18]:
!python3 --version

Python 3.10.12


In [19]:
!nvidia-smi

Tue Sep 10 07:40:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1060        Off |   00000000:01:00.0  On |                  N/A |
| N/A   70C    P2             26W /   78W |    1498MiB /   6144MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Api

In [20]:
import time
import numpy as np
import os
import _pickle as cPickle
import numpy as np
import imagenet_stubs
from imagenet_stubs.imagenet_2012_labels import label_to_name as IMAGENET_CLASS_NAMES
import pycuda.autoinit
import pycuda.driver as drv
import numpy
from PIL import Image
import imagehash

from pycuda.compiler import SourceModule

PATH = "../data/cifar-10-batches-py/"
IMAGENET_PATH = "../data/Imagenet32_train/"

In [21]:
def unpickleImages(pairs):
    images = set()
    for elem in pairs:
        images.add(elem[0])
        images.add(elem[1])

    return list(images)

def load_batch(batchFile):
    f = open(PATH+batchFile, 'rb')
    
    dict_ = cPickle.load(f, encoding='latin1')
    images = dict_['data']
    labels = dict_['labels']
    imagearray = np.array(images)   #   (10000, 3072)
    labelarray = np.array(labels)   #   (10000,)
    
    return imagearray, labelarray

def load_classes():
    file = 'batches.meta'
    
    f = open(PATH+file, 'rb')
    dict = cPickle.load(f)
    return dict['label_names']

In [22]:
unpickleImages([(1,2, 0.85),(3,4,0.34)])

[1, 2, 3, 4]

In [23]:
CIFAR_IMAGES, CIFAR_LABELS = load_batch("data_batch_1")
CIFAR_CLASS_NAMES = load_classes()
CIFAR_IMAGES.shape

(10000, 3072)

In [24]:
mod = SourceModule("""

#include <cmath>

__global__ void ResizeAndGray1000ImagesInterpotedHash(float* image, float* hash, const int pixels, const int new_rows, 
                                        const int new_cols, const int amountOfImages, const int hashSize)
{

    int idx           = threadIdx.x + blockIdx.x * blockDim.x;
    float* pixelsValues = new float [new_rows * new_cols];

    const float originImageSize = 32;
    float xRatio = originImageSize / new_cols;
    float yRatio = originImageSize / new_rows;

    if ( xRatio < yRatio ) xRatio = yRatio;
    
    if (idx > amountOfImages) return;

    int imageStartIdx = pixels * pixels * 3 * idx;
    
    for( int i=0; i<new_rows; i++ )
    {
        for ( int j=0; j<new_cols; j++)
        {
            float xPixel = i*yRatio;
            float yPixel = j*xRatio;

            const int x_l = floor(xPixel);  // x lower
            const int y_l = floor(yPixel);
            const int x_h = ceil(xPixel);   // x higher
            const int y_h = ceil(yPixel);

            const float x_weight = xPixel - x_l;
            const float y_weight = yPixel - y_l;

            const int pixelToTakeA = imageStartIdx + x_l * 32 + y_l;
            const int pixelToTakeB = imageStartIdx + x_h * 32 + y_l;
            const int pixelToTakeC = imageStartIdx + x_l * 32 + y_h;
            const int pixelToTakeD = imageStartIdx + x_h * 32 + y_h;
            
            const float a = 0.2989 * image[pixelToTakeA] + 0.5870 * image[1024+pixelToTakeA] + 0.1140 * image[2048+pixelToTakeA];
            const float b = 0.2989 * image[pixelToTakeB] + 0.5870 * image[1024+pixelToTakeB] + 0.1140 * image[2048+pixelToTakeB];
            const float c = 0.2989 * image[pixelToTakeC] + 0.5870 * image[1024+pixelToTakeC] + 0.1140 * image[2048+pixelToTakeC];
            const float d = 0.2989 * image[pixelToTakeD] + 0.5870 * image[1024+pixelToTakeD] + 0.1140 * image[2048+pixelToTakeD];

            const float pixel = a * (1 - x_weight) * (1 - y_weight) 
              + b * x_weight * (1 - y_weight) +
              c * y_weight * (1 - x_weight) +
              d * x_weight * y_weight;
              
            pixelsValues[i * new_rows + j ] = pixel;
        }
    }

    for( int i=0; i<new_rows; i++ )
    {
        for ( int j=1; j<new_cols; j++)
        {
            pixelsValues[i*new_rows+j-1] < pixelsValues[i*new_rows+j] ? 
                hash[hashSize*idx+i*new_rows+j-1] = 1 : hash[hashSize*idx+i*new_rows+j-1] = 0;
        }
    }

    delete pixelsValues;

    __syncthreads();
}

""")

ResizeAndGray1000ImagesInterpotedHash = mod.get_function("ResizeAndGray1000ImagesInterpotedHash")

In [26]:
mod = SourceModule("""

__global__ void ComparePairs(float* pairs, float* signatures, float* res, float threshold, float hash_size, int pairsElements)
{
    unsigned int x = blockIdx.x * blockDim.x + threadIdx.x;

    if(x > pairsElements) return;

    const int bitsNumbers = hash_size * hash_size / 8;
    const int imageIdx1 = pairs[x*2];
    const int imageIdx2 = pairs[x*2+1];
    unsigned int hammingDistance = 0;
    
    for(int i=0; i<bitsNumbers; i++)
    {
        unsigned int common = (unsigned int)signatures[imageIdx1*bitsNumbers+i] ^ (unsigned int)signatures[imageIdx2*bitsNumbers+i];

         while ( common>0 )
         {
            unsigned int val = common&1;
            common>>=1;
            hammingDistance += val;
        }
    }

    float similarity = (hash_size * hash_size - hammingDistance) / ( hash_size * hash_size );

    if ( similarity >= threshold )
    {
        res[x] = similarity;
    }
}

""")

ComparePairs = mod.get_function("ComparePairs")

In [27]:
from typing import List, Dict, Tuple

class cuPerceptual:
    def __init__(self, size, hash_size, images):
        self.buckets = []
        self.bandsNumber = size
        self.rows = int(hash_size**2 / self.bandsNumber)
        self.hash_buckets_list: List[Dict[int, List[str]]] = [dict() for _ in range(self.bandsNumber)]
        self.signatures = numpy.zeros([len(images), hash_size // 8 * hash_size]).astype(numpy.float32)
        self.hash_size = hash_size
        self.images = images;

        for i in range(size):
            self.buckets.append({})

    def calculate_signature_np(self, image_file: np.array, hash_size: int) -> np.ndarray:
        pil_image = Image.fromarray(image_file).convert("L").resize(
                            (hash_size+1, hash_size),
                            Image.BILINEAR)
        dhash = imagehash.dhash(pil_image, hash_size)
        signature = dhash.hash.flatten()
        pil_image.close()
        return signature
            
    def fill_buckets(self, signature, idx):
        for i in range(self.bandsNumber):
            signature_band = signature[i*self.rows:(i+1)*self.rows]
            signature_band_bytes = signature_band.tobytes()
            if signature_band_bytes not in self.hash_buckets_list[i]:
                self.hash_buckets_list[i][signature_band_bytes] = list()
            self.hash_buckets_list[i][signature_band_bytes].append(idx)

    def compute_hash(self):
        idx = 0;
        for image in self.images:
            signature = self.calculate_signature_np(image.T, self.hash_size)
            self.signatures[idx] = np.packbits(signature.astype(np.uint8))
            self.fill_buckets(signature, idx)
            idx+=1
        return self.signatures
    
    def compute_candidate(self):
        candidate_pairs = []

        for hash_buckets in self.hash_buckets_list:
            for hash_bucket in hash_buckets.values():
                if len(hash_bucket) > 1:
                    hash_bucket = sorted(hash_bucket)
                    for i in range(len(hash_bucket)):
                        for j in range(i+1, len(hash_bucket)):
                            candidate_pairs.append(hash_bucket[i])
                            candidate_pairs.append(hash_bucket[j])
        return candidate_pairs

    def check_candidate(self, candidate_pairs, threshold=0.9):
        pairsNumpy = numpy.asarray(candidate_pairs, dtype=np.float32)
        signaturesReshaped = self.signatures.reshape(-1)

        near_duplicates = list()
        
        for i in range(0, pairsNumpy.shape[0], 1000000):
            resSimilarity = numpy.zeros([500000]).astype(numpy.float32)
            ComparePairs(drv.In(pairsNumpy), drv.In(signaturesReshaped), drv.Out(resSimilarity), 
                         np.float32(threshold), np.float32(self.hash_size), np.int32(pairsNumpy.shape[0]//2),
                         block=(1000,1,1), grid=(500,1,1))

            for j in range (0, 1000000, 2):
                if i+j >= pairsNumpy.shape[0]:
                    break;
                if resSimilarity[j//2] > 0:
                    near_duplicates.append( ( int(pairsNumpy[i+j]), int(pairsNumpy[i + j + 1]), resSimilarity[j//2] ) )

        # Sort near-duplicates by descending similarity and return
        near_duplicates.sort(key=lambda x:x[2], reverse=True)
        return near_duplicates

In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [29]:
torch.set_default_device('cuda:0') 

In [30]:
class Net2(nn.Module):
    def __init__(self, bits):
        super().__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.hash1 = nn.Linear(256, bits)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(bits, 10)

    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.hash1(x)
        hash_= self.sigmoid(x);
        x = self.fc2(x)
        return x, hash_

HASH_SIZE = 64
net2 = Net2( HASH_SIZE )
net2.to("cuda:0")

Net2(
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (hash1): Linear(in_features=256, out_features=64, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [31]:
from torchvision import models

resnet50 = models.resnet50(pretrained = True) # weights=ResNet50_Weights.DEFAULT
resnet50 = resnet50.to("cuda:0")
resnet50.fc = torch.nn.Identity()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
net2 = Net2( HASH_SIZE ).to("cuda:0")

In [33]:
model2 = nn.Sequential(resnet50, net2)

In [34]:
for param in model2.parameters():
    param.requires_grad = False

In [35]:
PATH = "../2.NeuralHashing/mainModel.pth"
model2.load_state_dict(torch.load(PATH))
model2 = model2.eval()

In [36]:
from typing import List, Dict, Tuple

class cuNN:
    def __init__(self, size, hash_size, images, size_images ):
        self.buckets = []
        self.bandsNumber = size
        self.rows = int(hash_size**2 / self.bandsNumber)
        self.hash_buckets_list: List[Dict[int, List[str]]] = [dict() for _ in range(self.bandsNumber)]
        self.signatures = numpy.zeros([len(images), hash_size // 8 * hash_size]).astype(numpy.float32)
        self.hash_size = hash_size
        self.images = images;
        self.size_ = size_images

        for i in range(size):
            self.buckets.append({})
            
    def fill_buckets(self, signature, idx):
        for i in range(self.bandsNumber):
            signature_band = signature[i*self.rows:(i+1)*self.rows]
            signature_band_bytes = signature_band.tobytes()
            if signature_band_bytes not in self.hash_buckets_list[i]:
                self.hash_buckets_list[i][signature_band_bytes] = list()
            self.hash_buckets_list[i][signature_band_bytes].append(idx)

    def compute_hash(self):
        
        tensor_image = torch.FloatTensor(self.images).reshape(self.size_,3,32,32).to("cuda:0")
        label, hash_ = model2(tensor_image)
        signatures = hash_.cpu().detach().numpy().astype(np.int8)

        idx = 0
        for signature in signatures:
            self.signatures[idx] = np.packbits( signature ).astype(np.float32)
            self.fill_buckets(signature, idx)
            idx+=1
        return self.signatures
    
    def compute_candidate(self):
        #candidate_pairs = set()
        list1 = []
        list2 = []

        for hash_buckets in self.hash_buckets_list:
            for hash_bucket in hash_buckets.values():
                if len(hash_bucket) > 1:
                    hash_bucket = sorted(hash_bucket)
                    for i in range(len(hash_bucket)):
                        for j in range(i+1, len(hash_bucket)):
                            #candidate_pairs.add(
                            #   tuple([hash_bucket[i], hash_bucket[j]])
                            #)
                            #list1.append(self.signatures[hash_bucket[i]]) 
                            #list2.append(self.signatures[hash_bucket[j]]) 
                            list1.append(hash_bucket[i]) 
                            list2.append(hash_bucket[j]) 
        #return candidate_pairs
        return list1, list2
    
    def check_candidate(self, candidate_pairs, threshold=0.9):
        near_duplicates = list()
        for cpa, cpb in candidate_pairs:
            item_cpa = np.array(self.signatures[cpa]).astype(np.uint8)
            item_cpb = np.array(self.signatures[cpb]).astype(np.uint8)
            hd = sum(np.bitwise_xor(
                    np.unpackbits(item_cpa), 
                    np.unpackbits(item_cpb)
            ))
            similarity = (self.hash_size**2 - hd) / self.hash_size**2
            if similarity > threshold:
                near_duplicates.append((cpa, cpb, similarity))
                
        # Sort near-duplicates by descending similarity and return
        near_duplicates.sort(key=lambda x:x[2], reverse=True)
        return near_duplicates

In [38]:
nnLSH = cuNN(8, 8, CIFAR_IMAGES, len(CIFAR_IMAGES))
signatures = nnLSH.compute_hash()

print(signatures.shape)

pairs = nnLSH.compute_candidate()

print(len(pairs))
#duplicates = nnLSH.check_candidate(pairs)

# get images from duplicates
DUPLICATED_IMAGES = unpickleImages(duplicates)
#DUPLICATED_IMAGES=CIFAR_IMAGES
perceptualLSH = cuPerceptual(8, 8,DUPLICATED_IMAGES)
perceptualLSH.compute_hash()
pairs2 = perceptualLSH.compute_candidate()
duplicates2 = perceptualLSH.check_candidate(pairs2)

#calculate metrics
## size = len(self.images);

/usr/local/lib/python3.10/dist-packages/torch/utils/_device.py:78: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return func(*args, **kwargs)


(10000, 8)


KeyboardInterrupt: 

In [60]:
images = CIFAR_IMAGES.reshape(10000,3,32,32)
print(images.shape)

perceptualLSH = cuPerceptual(8, 8,images)
perceptualLSH.compute_hash()
pairs3 = perceptualLSH.compute_candidate()
duplicates3 = perceptualLSH.check_candidate(pairs3, 0.9)

# get images from duplicates
#DUPLICATED_INDICES2 = unpickleImages(duplicates3)
#DUPLICATED_IMAGES2 = np.array(CIFAR_IMAGES)[DUPLICATED_INDICES2]

(10000, 3, 32, 32)


LogicError: cuFuncSetBlockShape failed: invalid resource handle

In [50]:
# Treshold wwazny zeby odfiltrowac nie duplikaty
# Porownywanie duplikatow zdjec - przy podwojnym filtrowaniu ???
# 6 milionow par potencjalych przy sieci neuronowej
# Pierw musi chodzic PyCUDA poniej doperio cos PyTorch, zeby nie bylo przeklama w dostepie do cuda

In [22]:
print(len(DUPLICATED_IMAGES2))
print(CIFAR_IMAGES.shape)

nnLSH = cuNN(8, 8,DUPLICATED_IMAGES2, len(DUPLICATED_IMAGES2))
signatures2 = nnLSH.compute_hash()
print(signatures2.shape)

pairs4_1, pairs4_2 = nnLSH.compute_candidate()
print( len(pairs4_1), len(pairs4_2) )
#print(pairs4[0])
#duplicates4 = nnLSH.check_candidate(pairs4)

#calculate metrics
pairs4_1

5182
(10000, 3072)


/usr/local/lib/python3.10/dist-packages/torch/utils/_device.py:78: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return func(*args, **kwargs)


(5182, 8)
80198218 80198218


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [23]:
#list1.append(self.signatures[hash_bucket[i]]) 
                            #list2.append(self.signatures[hash_bucket[j]]) 
images_cuda = torch.IntTensor(CIFAR_IMAGES)
cpa = torch.IntTensor(pairs4_1).to("cuda:0")
cpb = torch.IntTensor(pairs4_2).to("cuda:0")

In [54]:
images_cuda[cpa[0]].numpy().astype(np.uint8)

array([ 59,  43,  50, ..., 140,  84,  72], dtype=uint8)

In [58]:
res = []
for i in range (0, 100 ):
    hd = sum(np.bitwise_xor(
                    np.unpackbits(images_cuda[cpa[i]].numpy().astype(np.uint8)), 
                    np.unpackbits(images_cuda[cpb[i]].numpy().astype(np.uint8))
            ))
    similarity = (8**2 - hd) / 8**2
    if similarity > 0.9:
        res.append((cpa[i].item(), cpb[i].item(), similarity.item(0)))

In [59]:
res

[]

In [39]:
cpb[0]

tensor(1, device='cuda:0', dtype=torch.int32)

In [ ]:
duplicates4

In [ ]:
from matplotlib import pyplot as plt

num_row = 6
num_col = 6
col_idx = 0
row_idx = 0

idx_start = 0;

fig, axes = plt.subplots(num_row, num_col, figsize=(3*num_col,3*num_row))
for i in range(0, num_row*num_col//2, 1):
    if col_idx >= num_col -1:
        col_idx = 0
        row_idx += 1
    
    ax = axes[row_idx, col_idx]
    ax.imshow(CIFAR_IMAGES[duplicates4[i + idx_start ][0]].reshape(3,32,32).T)
    ax.set_title(f"""Label: {duplicates4[i + idx_start ][0]}, {CIFAR_LABELS[duplicates4[i + idx_start ][0]]}, {duplicates4[i + idx_start][2]}""")

    col_idx += 1
    ax = axes[row_idx, col_idx]
    ax.imshow(CIFAR_IMAGES[duplicates4[i + idx_start][1]].reshape(3,32,32).T)
    ax.set_title(f"""Label: {duplicates4[i + idx_start][1]}, {CIFAR_LABELS[duplicates4[i + idx_start][1]]}, {duplicates4[i + idx_start][2]}""")
    col_idx += 1

plt.tight_layout()
plt.xticks([])
plt.yticks([])

plt.show()